In [18]:
%pwd

'e:\\Workspace\\End-to-End-Mlops'

In [19]:
import os

In [22]:
os.chdir("E:/Workspace/End-to-End-Mlops")

In [23]:
%pwd

'E:\\Workspace\\End-to-End-Mlops'

In [24]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml,create_directories


In [25]:
# Entity Config

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
   root_dir: Path
   train_data_path: Path
   test_data_path: Path
   model_name: str
   n_estimators: int
   learning_rate: float
   max_depth: int
   target_column: str


In [30]:
class ConfigurationManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH,schema_file_path=SCHEMA_FILE_PATH,para_file_path=PARAMS_FILE_PATH):

        self.config = read_yaml(config_file_path)
        self.schema = read_yaml(schema_file_path)
        self.params = read_yaml(para_file_path)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self)->ModelTrainerConfig:

        config=self.config.model_trainer
        params=self.params.XGB_MODEL
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        return ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            n_estimators=params.n_estimators,
            learning_rate=params.learning_rate,
            max_depth=params.max_depth,
            target_column=schema.name
        )



In [31]:
import numpy as np
import pandas as pd
from src.mlProject.logging import logger
from xgboost import XGBRegressor
import joblib

In [32]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train_model(self):
        train_data=pd.read_csv(self.config.train_data_path)
        test_data=pd.read_csv(self.config.test_data_path)

        train_x=train_data.drop([self.config.target_column],axis=1)
        test_x=test_data.drop([self.config.target_column],axis=1)
        train_y=train_data[self.config.target_column]
        test_y=test_data[self.config.target_column]

        xgb_lr=XGBRegressor(n_estimators=self.config.n_estimators,learning_rate=self.config.learning_rate,max_depth=self.config.max_depth)

        xgb_lr.fit(train_x,train_y)
        logger.info("Model trained successfully")
        
        joblib.dump(xgb_lr, os.path.join(self.config.root_dir, self.config.model_name))



In [33]:
try:
    config=ConfigurationManager()
    model_trainer_config=config.get_model_trainer_config()
    Model_trainer=ModelTrainer(config=model_trainer_config)
    Model_trainer.train_model()
    logger.info("Training completed successfully")
except Exception as e:
    logger.info("An error occurred during model training")
    raise e

--- Logging error ---
Traceback (most recent call last):
  File "e:\ML_Project\ml_project\lib\logging\__init__.py", line 1082, in emit
    stream.write(msg + self.terminator)
AttributeError: 'str' object has no attribute 'write'
Call stack:
  File "e:\ML_Project\ml_project\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "e:\ML_Project\ml_project\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\HP PC\AppData\Roaming\Python\Python39\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\HP PC\AppData\Roaming\Python\Python39\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\HP PC\AppData\Roaming\Python\Python39\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.start()
  File "C:\Users\HP PC\AppData\Roaming\Python\Python39\site-packages\tornado\platform\asyncio.py", lin